Open Loop Simulation

In [38]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import os

SIM_DIR = "out/sim"
CSV_OUTPUT = f"{SIM_DIR}/measurement.ChannelGroup_0_linear.csv"
SIMER_IMAGE = "ghcr.io/boschglobal/dse-simer:latest"
MDF_FILE = "measurement.mf4"


Drag and Drop Widget for CSV

In [39]:
import ipywidgets as widgets
from IPython.display import display

target_folder = "./csv_input" 
os.makedirs(target_folder, exist_ok=True)

# file uploader widget
uploader = widgets.FileUpload(
    accept='.csv', 
    multiple=False 
)

def handle_upload(change):
    if uploader.value:  # Check if a file has been uploaded
        for uploaded_file in uploader.value:
            file_name = uploaded_file['name']  # Get the file name
            file_content = uploaded_file['content']  # Get the file content
            # Save the file to the target folder
            file_path = os.path.join(target_folder, "input__value_set.csv")
            with open(file_path, 'wb') as f:
                f.write(file_content)
            print(f"File '{file_name}' has been uploaded and copied to '{target_folder}'.")

# Attach the function to the widget's change event
uploader.observe(handle_upload, names='value')

# Display the widget
print(f"Drag and drop your CSV file below. It will be saved to: {target_folder}")
display(uploader)


Drag and drop your CSV file below. It will be saved to: ./csv_input


FileUpload(value=(), accept='.csv', description='Upload')

Notebook Controller

In [40]:
from asammdf import MDF

def run_command(cmd):
    cmd_str = ' '.join(cmd)
    print(f"Running: {cmd_str}")
    result = os.system(cmd_str)
    if result != 0:
        raise RuntimeError(f"Command failed: {cmd_str}")
    print(f"Command executed successfully.")

def run_simulation():
    os.environ["TASK_X_REMOTE_TASKFILES"] = "1"
    os.environ["SIMER_IMAGE"] = SIMER_IMAGE

    # Build task
    run_command(["task", "-y", "build"])

    # Run simulation
    sim_cmd = [
        "docker", "run", "--rm",
        "-v", f"{os.getcwd()}/out/sim:/sim", SIMER_IMAGE,
        "-env", f"linear:MEASUREMENT_FILE=/sim/{MDF_FILE}",
        "-stepsize", "0.0005", "-endtime", "0.005"
    ]
    run_command(sim_cmd)
    print("Simulation complete.")

    # Convert MDF to CSV
    convert_to_csv(SIM_DIR, MDF_FILE, CSV_OUTPUT)
    print(f"Converted MDF to CSV: {CSV_OUTPUT}")

def convert_to_csv(sim_dir, mdf_file, csv_output):
    #Convert MDF file to CSV using asammdf
    in_file = os.path.join(sim_dir, mdf_file)
    out_file = csv_output
    mdf = MDF(in_file)
    mdf.export(fmt="csv", filename=out_file)

Run the Simulation

In [41]:
try:
    run_simulation()
    print("Simulation executed successfully.")
except Exception as e:
    print(f"Error in simulation: {e}")


Running: task -y build
The task you are attempting to run depends on the remote Taskfile at "https://raw.githubusercontent.com/boschglobal/dse.fmi/refs/tags/v1.1.15/Taskfile.yml".
--- Make sure you trust the source of this Taskfile before continuing ---
Continue? [assuming yes]
CURL https://github.com/boschglobal/dse.modelc/releases/download/v2.1.15/ModelC-2.1.15-linux-amd64.zip -> downloads/ModelC-2.1.15-linux-amd64.zip
CURL https://github.com/boschglobal/dse.fmi/releases/download/v1.1.15/Fmi-1.1.15-linux-amd64.zip -> downloads/Fmi-1.1.15-linux-amd64.zip


task: [dse:download-file:https://github.com/boschglobal/dse.modelc/releases/download/v2.1.15/ModelC-2.1.15-linux-amd64.zip-downloads/ModelC-2.1.15-linux-amd64.zip] echo "CURL https://github.com/boschglobal/dse.modelc/releases/download/v2.1.15/ModelC-2.1.15-linux-amd64.zip -> downloads/ModelC-2.1.15-linux-amd64.zip"
task: [dse:download-file:https://github.com/boschglobal/dse.modelc/releases/download/v2.1.15/ModelC-2.1.15-linux-amd64.zip-downloads/ModelC-2.1.15-linux-amd64.zip] mkdir -p $(dirname downloads/ModelC-2.1.15-linux-amd64.zip)
task: [dse:download-file:https://github.com/boschglobal/dse.modelc/releases/download/v2.1.15/ModelC-2.1.15-linux-amd64.zip-downloads/ModelC-2.1.15-linux-amd64.zip] curl --retry 5  -fL https://github.com/boschglobal/dse.modelc/releases/download/v2.1.15/ModelC-2.1.15-linux-amd64.zip -o downloads/ModelC-2.1.15-linux-amd64.zip
task: [dse:download-file:https://github.com/boschglobal/dse.fmi/releases/download/v1.1.15/Fmi-1.1.15-linux-amd64.zip-downloads/Fmi-1.1

FMU Fetch/Extract https://github.com/boschglobal/dse.fmi/releases/download/v1.1.15/Fmi-1.1.15-linux-amd64.zip/Fmi-1.1.15-linux-amd64/examples/fmu/linear/fmi2/linear.fmu -> sim/model/linear/fmu
UNZIP FILE downloads/Fmi-1.1.15-linux-amd64.zip/Fmi-1.1.15-linux-amd64/examples/fmu/linear/fmi2/linear.fmu -> fmus/linear.fmu
Archive:  downloads/Fmi-1.1.15-linux-amd64.zip
  inflating: fmus/linear.fmu         
Current directory 
SIM Model input -> sim/model/input
/home/kki8kor/dse.sdp/examples/openloop
UNZIP DIR fmus/linear.fmu/ -> sim/model/linear/fmu
Archive:  fmus/linear.fmu
   creating: sim/model/linear/fmu/resources/
 extracting: sim/model/linear/fmu/resources/.keepme  
   creating: sim/model/linear/fmu/binaries/
   creating: sim/model/linear/fmu/binaries/linux64/
  inflating: sim/model/linear/fmu/binaries/linux64/fmu2linear.so  UNZIP FILE downloads/ModelC-2.1.15-linux-amd64.zip/ModelC-2.1.15-linux-amd64/examples/csv/lib/libcsv.so -> sim/model/input/lib/libcsv.so

  inflating: sim/model/lin

Running FMI Toolset command: gen-mcl
Options:
  fmu            : model/linear/fmu
  log            : 4
  mcl            : model/linear/lib/libfmimcl.so
  outdir         : model/linear/data
  platform       : linux-amd64
Reading FMU Desciption (model/linear/fmu/modelDescription.xml)
Creating Model YAML: linear (model/linear/data/model.yaml)
task: [dse:fmi:gen-signalgroup] docker run --rm -v /home/kki8kor/dse.sdp/examples/openloop/out/sim:/sim ghcr.io/boschglobal/dse-fmi:1.1.15 gen-signalgroup --input model/linear/fmu/modelDescription.xml --output 'model/linear/data/signalgroup.yaml'
Running FMI Toolset command: gen-signalgroup
Options:
  input          : model/linear/fmu/modelDescription.xml
  log            : 4
  output         : model/linear/data/signalgroup.yaml
Reading file: model/linear/fmu/modelDescription.xml
Appending file: model/linear/data/signalgroup.yaml
task: [build] mkdir -p 'sim/data'
task: [build] cp /home/kki8kor/dse.sdp/examples/openloop/simulation.yaml 'sim/data/simul

Command executed successfully.
Running: docker run --rm -v /home/kki8kor/dse.sdp/examples/openloop/out/sim:/sim ghcr.io/boschglobal/dse-simer:latest -env linear:MEASUREMENT_FILE=/sim/measurement.mf4 -stepsize 0.0005 -endtime 0.005
1) Version: 2.1.6
1) Load YAML File: data/simulation.yaml
2) Version: 2.1.6
2) Platform: linux-amd64
2) Time: 2025-00-17 08:41:52 (UTC)
2) Host: a009f096737d
2) CWD: /sim
2) Load YAML File: data/simulation.yaml
3) Version: 2.1.6
3) Platform: linux-amd64
3) Time: 2025-00-17 08:41:52 (UTC)
3) Host: a009f096737d
3) CWD: /sim
3) Load YAML File: data/simulation.yaml
2) Load YAML File: model/input/data/model.yaml
3) Load YAML File: model/linear/data/model.yaml
2) Load YAML File: model/input/data/signalgroup.yaml
3) Load YAML File: model/linear/data/signalgroup.yaml
3) Simulation Parameters:
3)   Step Size: 0.000500
3)   End Time: 0.005000
3)   Model Timeout: 60.000000
3)   Sim Path: (null)
3) Transport:
3)   Transport: redispubsub
3)   URI: redis://localhost:6379
3

Load and Visualize Results

In [ ]:
# Load CSV
if Path(CSV_OUTPUT).exists():
    results = pd.read_csv(CSV_OUTPUT)
    print("Simulation Results Loaded.")
else:
    raise FileNotFoundError(f"Output file not found: {CSV_OUTPUT}")

# Plot results
plt.figure(figsize=(10, 5))
for col in results.columns[1:]:
    plt.plot(results['timestamps'], results[col], label=col)

plt.title("Simulation Results")
plt.xlabel("Time")
plt.ylabel("Values")
plt.legend()
plt.grid()
plt.show()

